In [1]:
import ast, linsolve

# `linsolve` Quickstart Guide

`linsolve` is a module providing high-level tools for (linearizing and) performing a least-squares optimization on a system of equations.

In [2]:
import numpy as np
import linsolve

## Solving Linear Least-Squares Systems of Equations with `LinearSolver`

In `linsolve`, data is expressed as python dictionary object where equations are passed in as a dictionary where which maps strings containing all unknowns (the RHS of the system) to numbers (the LHS of the system). The strings are parsed according to python syntax and all variables are automatically identified (as long as they are valid python variable names) and solved for.

For background, see <https://en.wikipedia.org/wiki/Linear_least_squares_(mathematics)>

As an example, here is how you would the following system

$3 x + 4 y = 11 \\ -x -3y = -7$

In [3]:
import numpy as np
import linsolve

#Set up dictionary of equations
data = {'3*x+4*y': 11.0, '-1*x-3*y': -7.0}

#Build solver
ls = linsolve.LinearSolver(data)

#Execute solver
sol = ls.solve()
print(sol)

{'x': 1.0, 'y': 2.0}


### Array Systems

Often, it its useful to solve the same system of equations for many different "measurements" on the LHS of the system of equations. In this case, the dictionary of equations can map strings to $N$-dimensional numpy arrays.

In [ ]:
data = {'3*x+4*y': np.array([11.0,22.0]), '-1*x-3*y': np.array([-7.0,-14.0])}
ls = linsolve.LinearSolver(data)
sol = ls.solve()
print(sol)

### Constants

While all terms that evaluate to valid python variables in the dictionary keys are assumed to be variables, it is possible to designate some of them as constants. They are passed `LinearSolver` via its `kwargs`. Constants can be expressed either as single numbers or as arrays that can broadcast according to `numpy` rules to the shape of the data, allowing the equations to change between solves.

In [ ]:
data = {'3*x+a*y': np.array([11.0,22.0]), 'b*x-3*y': np.array([-7.0,-14.0])}
ls = linsolve.LinearSolver(data, a=4, b=np.array([-1,-2]))
sol = ls.solve()
print(sol)

Any variable names that are not assigned values are assumed to be parameters in need of solutions.  However, `linsolve` being a linear solver, if more than one variable per term is unknown (i.e. if the expression is non-linear), `linsolve` will raise an `AssertionError`.

In [ ]:
try:
    ls = linsolve.LinearSolver({'a*x':6., 'b*x':8.}) # raises an AssertionError because 'a' and 'b' are not defined.
except(AssertionError):
    print('AssertionError encountered.')

### Complex Conjugation 

Complex conjugation of variables is accomplished via trailing underscore in the variable name.

In [ ]:
data = {'x': 1.0+1.0j, 'y_': 1.0+1.0j}
ls = linsolve.LinearSolver(data)
sol = ls.solve()
print(sol)

### Weights

When a system of equations is overdetermined (as is usually the case in data analysis), `linsolve` lets you apply different weights to each equation. The default behavior is to weight all equations equally.

In [ ]:
data = {'1*x': 2.0, 'x': 1.0}
ls = linsolve.LinearSolver(data)
sol = ls.solve()
print(sol)

However, you can provide weights through the `wgts` keyword argument. Weights are multiplied by the data, so in standard inverse covariance weighting they can be thought of as $1/\sigma^2$ where $\sigma$ is the noise on that equation.

In [ ]:
data = {'1*x': 2.0, 'x': 1.0}
wgts = {'1*x': 1.0, 'x': .5}
ls = linsolve.LinearSolver(data, wgts=wgts)
sol = ls.solve()
print(sol)

This solution, $x = 5/3$ is twice $(1/W^2)$ as close to $x=2$ as $x=1$. This is because it minimizes $\chi^2$, defined as

$\chi^2 = \sum_i \frac{(d_i - m_i)^2}{\sigma_i^2} = \sum_i W_i^2 (d_i - m_i)^2$

where $i$ indexes over equations, $d_i$ is the data (RHS), $m_i$ is the model (LHS evaluated at `sol`), and $W_i$ are the weights. We can calculate $\chi^2$ with this handy function:

In [ ]:
print(ls.chisq(sol))

Weighting allows for the implementation of inverse variance weighting, but assumes that the noise in each equation/measurement is independent (i.e. it does not allow for inverse *covariance* weighting).

## Linearizing Systems of Equations 

`linsolve` is also able to solve very basic nonlinear systems of equations using either `LogProductSolver` or `LinProductSolver`.

For background, see <https://en.wikipedia.org/wiki/Non-linear_least_squares>

### `LogProductSolver` 

`LogProductSolver` can tackle a special class of non-linear equations where all expressions are products. For example, we can solve

$ab = 2 \\ bc = 1 \\ ac = 2$

In [ ]:
data = {'a*b': 2.0, 'b*c': 1.0, 'a*c': 2.0}
lps = linsolve.LogProductSolver(data)
sol = lps.solve()
print(sol)

This is acheived by reducing the problem to a linear system of equations by taking the logarithm of both sides. In the case of an overdetermined system of equation, this can produce biased results that do not always minimize $\chi^2$. However, it is still useful as a starting guess for `LinProductSolver`.

Just as with `LinearSolver`, `LogProductSolver` can take array systems, constants passed as keyword arguments, complex conjugated variables, and equation weighting.

### `LinProductSolver`

`LinProductSolver` is a more general linearizer of equations. It can tackle equations that are sums (or differences) of products of variables (no quotients, powers, exponents, etc.). Parentheses are not allowed; the user is expected to expand manually. For example, `LinProductSolver` can tackle equations of the form:

$10ab + 3bc - 2ac = 32 \\ 2aa + 3bb - 2cc = -4 \\ -aa + 2bc + ac = 14$


It uses the [Gauss-Newton algorithm](https://en.wikipedia.org/wiki/Gauss%E2%80%93Newton_algorithm) to find a solution that minimzes $\chi^2$ (thought it may not be unique). This is an second-order iterative algorithm whose success depends on a decent initial guess `sol0`, formatted the same way as all `linsolve` solutions. The equation solved at each step is a Taylor expansion about `sol0`, which for the first example equation above gives

$10(a_0+\Delta a)(b_0+\Delta b) + 3(b_0 + \Delta b)(c_0 + \Delta c) - 2(a_0 + \Delta a)(c_0 + \Delta c) = 32 \\ 10a\Delta b + 10b\Delta a + 3b\Delta c + 3c \Delta b - 2a\Delta c - 2c\Delta a + \mathcal{O}(\Delta^2) = 32 - 10a_0b_0 + 3b_0c_0 - 2a_0c_0$ 

`LinProductSolver` then solves for all the $\Delta$ terms and updates `sol0` to `sol` accordingly. Just as with `LinearSolver`, `LinProductSolver` can take array systems, constants passed as keyword arguments, complex conjugated variables, and equation weighting.

In [ ]:
data = {'10*a*b + 3*b*c - 2*a*c': 32, 
        '2*a*a +3*b*b -2*c*c': -4,
        '-a*a + 2*b*c + a*c': 14}
sol0 = {'a': .9, 'b': 2.1, 'c': 3.2}
lps = linsolve.LinProductSolver(data, sol0)
meta, sol = lps.solve_iteratively()
print(sol)

`LinProductSolver.solve_iteratively()` produces both a solution dictionary in the standard `linsolve` format, but also produces `meta`, a metadata dictionary that contains:

* `conv_crit`: A convergence criterion (default `1e-10`) below which to stop iterating. Converegence is measured $\ell^2$-norm of the change in the solution of all the variables since the previous iteration divided by the $\ell^2$-norm of the solution itself.
* `chisq`: $\chi^2$ as calculated above for the final iteration
* `iter`: the number of iterations ran. The iterative solver runs until until either convergence or `maxiter` is hit (default 50).

In [ ]:
print(meta)

#### One Caveat About Variable Names in `LinProductSolver`

One should avoid having variables that differ from each other by only a starting `d`, e.g. `dram` and `ram`. Internally, `LinProductSolver` performs a Taylor expansion, treating the variables in `sol0` as constants and creating a corresponding set of variables representing differential adjustments that have a `d` prepended.